In [1]:
import momi

# store momi logging output in logfile "tutorial.log"
import logging
logging.basicConfig(level=logging.INFO,
                    filename="tutorial.log", filemode="w") 
# use level=logging.DEBUG for more verbose output 

Use the **`help()`** function to view documentation.

In [2]:
help(momi)

Help on package momi:

NAME
    momi

DESCRIPTION
    momi (MOran Models for Inference) is a python package for computing the site frequency spectrum,
    a summary statistic commonly used in population genetics, and using it to infer demographic history.
    
    Please refer to examples/tutorial.ipynb for usage & introduction.

PACKAGE CONTENTS
    compute_sfs
    concatenate_datasets
    confidence_region
    convolution
    data (package)
    demo_model
    demo_plotter
    demography
    einsum2 (package)
    events
    fstats
    likelihood
    math_functions
    moran_model
    optimizers
    parse_ms
    size_history
    util
    vcf2momi

FILE
    /home/jack/.local/lib/python3.6/site-packages/momi/__init__.py




# Creating demographies

momi uses a syntax based on the program ms by Richard Hudson.
A demography is specified as a sequence of events.
Time is measured going backwards from the present (t=0) to the past (t>0).

There are 4 kinds of events:
* **('-en', t, i, N)**
    * At time t, population i has its scaled population size set to N, and growth rate set to 0.
* **('-eg', t, i, g)**
    * At time t, population i has exponential growth rate g (so for s>t, $N(s) = N(t) e^{(s-t)  g}$)
* **('-ej', t, i, j)**
    * At time t, all lineages in population i move into j.
* **('-ep', t, i, j, p_ij)**
    * At time t, each lineage in i moves into j with probability p_ij.

Note **-en,-eg,-ej** are flags from ms, while **-ep** replaces the flag **-es** in ms.
All parameters are scaled like in **ms**. That is, the population sizes are assumed to be rescaled by a "reference" size N_ref (e.g. 10,000), and time is scaled so there are 4*N_ref generations per unit time.

Unlike ms, populations can be labeled by arbitrary strings. We illustrate this in the example below.

See **`help(momi.Demography)`** or **`help(momi.Demography.__init__)`** for more details.

### An example demography

Now we consider a function for creating a demography from some parameters. This demography includes 2 sampled populations, **'yri'** and **'chb'**, with 14 and 10 alleles, respectively. The demography also involves admixture with a third population, **'nea'**.

Note that for math functions, we use `autograd.numpy`, which is a drop-in replacement for `numpy` that allows for **automatic differentiation**. We will this in more detail in Section **"Automatic differentiation and autograd"**. 

In [3]:
import autograd.numpy as np

def demo_func(pulse_t, pulse_p,
              dt_join_chb_yri, dt_join_yri_nea,
              N_chb_bottom, N_chb_top,):
    events = []

    # "chb" receives a pulse from "nea"
    # at time pulse_t and strength pulse_p
    events.append(('-ep', pulse_t, 'chb', 'nea', pulse_p))

    # going backwards-in-time by dt_join_chb_yri,
    # "chb" joins onto "yri"
    t_join_chb_yri = pulse_t + dt_join_chb_yri
    events.append(("-ej", t_join_chb_yri, "chb", "yri"))

    # going backwards-in-time by dt_join_yri_nea,
    # "yri" joins onto "nea"
    t_join_yri_nea = t_join_chb_yri + dt_join_yri_nea
    events.append(("-ej", t_join_yri_nea, "yri", "nea"))

    # at the present, "chb" has population size N_chb_bottom
    events.append(("-en", 0, "chb", N_chb_bottom))

    # "chb" has exponentially growing population size, so that
    # it had size N_chb_top at time t_join_chb_yri
    growth_rate_chb = -np.log(N_chb_top / N_chb_bottom) / t_join_chb_yri
    events.append(("-eg", 0, "chb", growth_rate_chb))

    # create the demography from these events, and
    # specify "yri" and "chb" to have 14 and 10 alleles, respectively
    return momi.make_demography(
        events, sampled_pops=('yri','chb'), sampled_n=(14,10))

We set the "true" demography to have the following parameters.
We will simulate some data from this demography and try to infer it.

In [4]:
true_params = [.25, .03, .25, 1., 10., .1]
true_demo = demo_func(*true_params)

# Simulating and reading data

In this section we will simulate some data from the "true" demography.
We will save it as vcf, and then show how to read data into momi from
vcf format.

## Simulating data

We simulate 20 loci of length 500 Kb and save them as vcfs.

Note that the per-base mutation and recombination rates are in ms time
units, i.e. the mutation rate is the number of mutations per 4*N_ref generations.

In [5]:
import gzip
import os

per_base_mut_rate = .001
per_base_recom_rate = .001
bases_per_locus = int(5e5)
n_loci = 20
ploidy = 2

# create data directory if it doesn't exist
os.makedirs("data", exist_ok=True) 

In [6]:
# simulate 20 "chromosomes", saving each in a separate gzipped vcf file
for chrom in range(n_loci):
    with gzip.open("data/{}.vcf.gz".format(chrom), "wt") as outfile:
        true_demo.simulate_vcf(
            outfile,
            mutation_rate=per_base_mut_rate,
            recombination_rate=per_base_recom_rate,
            length=bases_per_locus,
            chrom_names=["chr{}".format(chrom)],
            ploidy=ploidy,
            random_seed=1234+chrom) 

## Reading data from vcf

We read the data into momi from the vcf files, using the function momi.SnpAlleleCounts.read_vcf().

Aside from the vcf file itself, this function has 2 important parameters:
1. **ind2pop**, a dict mapping samples to their populations
2. **ancestral_alleles**, which tells momi how to determine which allele is the ancestral one. This parameter can take 3 possible values:
  * If ancestral_alleles=True, then momi will read the ancestral allele from the "AA" INFO field. SNPs missing this field will be ignored.
  *  If ancestral_alleles=False or None, then momi will assume that REF is the ancestral allele. (This is the case for the simulated data).
  * If ancestral_alleles is the name of a population, then momi will treat that population as an outgroup, and use its allele as the ancestral one. (If the population contains both alleles, the SNP is ignored).

If you don't know the ancestral allele, use ancestral_alleles=False; you can later tell momi to use the folded SFS, which ignores whether an allele is ancestral or derived.

In [7]:
# Create dict mapping samples to populations
ind2pop = {}
for pop, n in zip(true_demo.sampled_pops, true_demo.sampled_n):
    for i in range(int(n / ploidy)):
        # in the vcf, the i-th individual
        # in population p was named "{p}_{i}"
        ind2pop["{}_{}".format(pop, i)] = pop

# No "AA" INFO field; use REF as the ancestral allele
ancestral_alleles = None

# Read in each vcf in a for loop
data = []
for chrom in range(n_loci):
    with gzip.open("data/{}.vcf.gz".format(chrom), "rt") as f:
        data.append(momi.SnpAlleleCounts.read_vcf(f, ind2pop,
                                                  ancestral_alleles))

# concatenate the 20 loci into a single dataset
data = momi.SnpAlleleCounts.concatenate(data)

## Saving data in custom json format

Use the dump() method to save the data in a custom json format,
which is quicker for momi to read then VCF.

Use momi.SnpAlleleCounts.load() to load data saved in this json format.

In [8]:
with gzip.open("data/momi_data.json.gz", "wt") as f:
    data.dump(f)

with gzip.open("data/momi_data.json.gz", "rt") as f:
    data2 = momi.SnpAlleleCounts.load(f)

assert data == data2

# Inference

To do inference, we first create a likelihood surface by passing the data
and demo_func to **momi.SfsLikelihoodSurface()**.

Other parameters include the mutation rate, locus length,
whether to use the folded SFS (recommended when the true ancestral allele is unknown),
and a "batch size" for computing SFS entries in batches (reduce this to decrease memory usage).

In [9]:
surface = momi.SfsLikelihoodSurface(
    data, demo_func,
    # set mut_rate=None if you don't know the mutation rate
    # can pass in lists for per-locus mut_rate/length
    mut_rate=per_base_mut_rate, length=bases_per_locus,
    # set folded=True if you don't know the ancestral allele
    folded=False,
    # decrease to reduce memory usage (compute SFS in smaller batches)
    batch_size=1000) 

To find the MLE, call the method **momi.SfsLikelihoodSurface.find_mle**.

This is essentially a wrapper around scipy.optimize.minimize, and takes many of the same arguments.
Here, we pass in the starting point for the parameter search, (lower, upper) bounds for each parameter,
and a callback function that prints every 5th step to STDOUT.

The default method used for searching the parameter space is "tnc", which works well for a small
number of parameters (roughly 20). For larger parameter spaces, the method "L-BFGS-B" is recommended.

Note that this only finds a local optimum, it is recommended to try many random start points to find
the true global maximum.

In [10]:
# define (lower,upper) bounds on the parameter space
bounds = [(.01,5.),
          (1e-100,.25),
          (.01,5.),
          (.01,5.),
          (.01, 100.),
          (.01, 100.)]

# pick a random start value for the parameter search
import random
x0 = [random.triangular(lower, upper, mode)
      for (lower, upper), mode in zip(bounds, [1, 0.1, 1, 1, 1, 1])] 

In [11]:
# callback function to print every 5th step to STDOUT
def callback(x):
    if x.iteration % 5 == 0:
        print ("iter  = {0}, kl-divergence = {1}, x = {2}".format(
            x.iteration, x.fun, x))

res = surface.find_mle(x0=x0, bounds=bounds, callback=callback,
                       # use method="L-BFGS-B" if >>20 parameters 
                       # use method="Nelder-Mead" or method="Powell" to avoid using gradients
                       method="tnc",
                       options={'maxiter':250})

iter  = 0, kl-divergence = 0.44196012155004366, x = [  0.8820758    0.22108438   0.5680652    1.61156889   0.47459644
  79.6846602 ]
iter  = 5, kl-divergence = 0.17734213359928547, x = [  6.82466812e-01   1.83935104e-01   1.00000000e-02   1.00000000e-02
   4.90925576e-01   7.47057526e+01]
iter  = 10, kl-divergence = 0.04967649324517103, x = [ 0.42619625  0.18224958  0.01        0.01        2.74017106  0.19570562]
iter  = 15, kl-divergence = 0.01239511662971747, x = [  4.83133654e-01   1.68026632e-01   1.00000000e-02   1.56268277e-01
   1.06762219e+01   1.29377722e-01]
iter  = 20, kl-divergence = 0.007732855244792322, x = [  0.32968246   0.1557145    0.14158693   0.49921019  18.549751
   0.03329411]
iter  = 25, kl-divergence = 0.004384463133041531, x = [  0.29011707   0.10802974   0.2005829    0.35720525  18.42539448
   0.03213004]
iter  = 30, kl-divergence = 0.003245021898788344, x = [  0.27540549   0.03520088   0.22314636   0.70488392  11.06873796
   0.08498184]
iter  = 35, kl-diverge

In [12]:
# info such as inferred x, whether search succeeded, number function/gradient evaluations, etc
res

     fun: 0.0031868809268584238
     jac: array([ -1.72553276e-06,   4.82393976e-06,   5.79543533e-06,
         2.59469476e-07,  -1.05387104e-07,  -2.09438317e-05])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 195
     nit: 37
  status: 1
 success: True
       x: array([  0.26323286,   0.02463737,   0.23913914,   0.9091638 ,
        10.45866508,   0.09441628])

In [13]:
# print out the inferred vs true parameters
import inspect
import pandas as pd
pd.DataFrame(list(zip(inspect.signature(demo_func).parameters.keys(),
                      true_params,
                      res.x)),
             columns=["Parameter", "Truth", "Inferred"])

,Parameter,Truth,Inferred
0,pulse_t,0.25,0.263233
1,pulse_p,0.03,0.024637
2,dt_join_chb_yri,0.25,0.239139
3,dt_join_yri_nea,1.00,0.909164
4,N_chb_bottom,10.00,10.458665
5,N_chb_top,0.10,0.094416


# Automatic diffentiation and autograd

momi uses the package **autograd** to compute gradients of the likelihood surface.
To ensure this works correctly, you should make sure demo_func() is compatible with autograd.

You can avoid using autograd by using a non-gradient based method in momi.SfsLikelihoodSurface.find_mle(),
such as method="Nelder-Mead" or method="Powell".
Alternatively, you can use momi.SfsLikelihoodSurface.find_mle(..., jac=False, ...) to have the optimizer
numerically approximate the gradient (but this is much slower than automatically computing it).
See help(momi.SfsLikelihoodSurface.find_mle) for more details.

Documentation for autograd can be found at https://github.com/HIPS/autograd/
Some do's and don'ts for autograd (from the autograd tutorial):
* Do use
    * Arithmetic operations `+,-,*,/,**`
    * Most of `numpy`'s functions
    * Most `numpy.ndarray` methods
    * Some `scipy` functions
    * Indexing and slicing of arrays `x = A[3, :, 2:4]`
    * Explicit array creation from lists `A = np.array([x, y])`
* Don't use
    * Assignment to arrays `A[0,0] = x`
    * Implicit casting of lists to arrays `A = numpy.sum([x, y])`, use `A = numpy.sum(np.array([x, y]))` instead.
    * `A.dot(B)` notation (use `numpy.dot(A, B)` instead)
    * In-place operations (such as `a += b`, use `a = a + b` instead)


# Other features 

## Computing individual SFS entries

To compute individual SFS entries under a demography,
first construct the entries ("configurations") to compute.
These are represented in the form $((a_0,d_0),(a_1,d_1),...)$
where there are $a_0$ ancestral and $d_0$ derived alleles in population 0, $a_1$ ancestral and $d_1$ derived alleles in population 1, etc.

Then call momi.expected_sfs(), as below.
See help(momi.expected_sfs) for options (e.g. folded SFS, sampling error).

In [14]:
# a list of configs (index0 == yri, index1 == chb)
configs = [( (14,0), (9,1), ), # 1 derived allele in chb
           ( (13,1), (10,0), ), # 1 derived allele in yri 
           ( (11,3), (9,1),) , # 3 derived in yri, 1 derived in chb
           ( (14,0), (0,10), ), # 0 derived in yri, all derived in chb
           ( (2,12), (10,0), ), # 12 derived in yri, 0 derived in chb 
           ( (2,12), (2,8), ), # 12 derived in yri, 8 derived in chb
          ]

configs = momi.config_array(("yri","chb"), configs)

# the expected number of SNPs with the given configs for mut_rate=1.0
momi.expected_sfs(true_demo, configs, mut_rate=1.0) 

array([  2.01612258e+00,   9.71262945e-01,   9.74375162e-04,
         2.70429603e-01,   5.54297456e-02,   1.44535809e-03])

## Coalescent statistics

`momi` can compute expected values of several coalescent statistics,
such as the TMRCA (time to most recent common ancestor)
and the total branch length of the genealogy.

In [15]:
eTmrca = momi.expected_tmrca(true_demo)
print("Expected TMRCA of all samples:", "\t", eTmrca)

eTmrca_chb = momi.expected_deme_tmrca(true_demo, 'chb')
print("Expected TMRCA of chb samples:", "\t", eTmrca_chb)

eL = momi.expected_total_branch_len(true_demo)
print ("Expected total branch length:", "\t", eL)

# See help(momi.expected_tmrca), etc. for more details.
# Advanced users can use momi.expected_sfs_tensor_prod()
# to compute these and many other summary statistics.

Expected TMRCA of all samples: 	 1.3124799948772297
Expected TMRCA of chb samples: 	 0.8067087315312199
Expected total branch length: 	 6.96253399983


You can also use autograd to compute the gradients of these statistics.

In [16]:
import autograd

# function mapping vector of parameters to the TMRCA of the corresponding demography
def tmrca_func(params):
    # equivalent to demo_func(params[0], params[1], params[2], ...)
    demo = demo_func(*params)
    # return expected TMRCA
    return momi.expected_tmrca(demo)

# use autograd.grad() to obtain the gradient function
tmrca_grad = autograd.grad(tmrca_func)

tmrca_grad(np.array(true_params))

array([ 0.28442909,  3.9289342 ,  0.83158351,  0.13903447,  0.00452414,
        0.55834569])

## Stochastic gradient descent

Stochastic gradient descent uses a random subset of SNPs to estimate the
gradient at each step. The expected value of the stochastic gradient
is the true gradient, but the stochastic gradient has added noise.

When there are many populations, the SFS will have many unique entries
and will be very large, so stochastic gradient descent allows quicker
exploration of the parameter space. It is particularly useful for
finding a good starting point before performing full optimization.

momi uses the ADAM algorithm to automatically adjust the stepsize
during the search, however the user must specify an initial stepsize.

momi also uses the SVRG algorithm to improve the accuracy of the
stochastic gradient. If the "svrg_epoch" parameter is set to a positive
integer k, then every k steps the optimizer will compute the full
likelihood (on all SNPs), and use this to reduce the variance of the
stochastic gradient.
(See papers for "Stochastic Variance Reduced Gradient".) 

In [17]:
# put parameters on log-scale (this is usually easier to optimize)
def demo_func2(*log_x):
    return demo_func(*np.exp(np.array(log_x)))

surface2 = momi.SfsLikelihoodSurface(
    data, demo_func2,
    mut_rate=per_base_mut_rate, length=bases_per_locus,
    folded=False)

# callback function to print every 5th step to STDOUT
def callback(x):
    if x.iteration % 50 == 0:
        print ("iter  = {0}, log-likelihood = {1}, x = {2}".format(
            x.iteration, x.fun, np.exp(x)))

res2 = surface2.stochastic_find_mle(
    x0=np.log(x0), snps_per_minibatch=10**4, stepsize=.01,
    num_iters=250, bounds=np.log(bounds),
    callback=callback,
    # compute full likelihood every 25 epochs
    # set svrg_epoch=-1 to avoid this
    svrg_epoch=25)

iter  = 0, log-likelihood = 6.3705451424422455, x = [  1.89126506   0.22258848   1.58947994   1.88686106  20.69464169
  81.32020079]
iter  = 50, log-likelihood = 4.6914536332021575, x = [  1.18914131   0.13219203   0.99623141   1.19894784  12.90858392
  50.44797898]
iter  = 100, log-likelihood = 3.920382615777373, x = [  0.83707938   0.0837325    0.69655534   0.88425705   8.79136047
  33.59674937]
iter  = 150, log-likelihood = 3.540377837139492, x = [  0.64761737   0.05983157   0.53620527   0.72512278   6.40401632
  23.58967058]
iter  = 200, log-likelihood = 3.3352740470683804, x = [  0.53883032   0.04660341   0.44431983   0.63125918   4.86706141
  17.05804193]
